# Full NLP and ML Pipeline for Document Classification
Based on following tutorials   
With Permission from Michale Harmon:
http://michael-harmon.com/blog/NLP.html   
https://github.com/mdh266/DocumentClassificationNLP/blob/master/NLP.ipynb   

Classification of text documents using sparse features:   
https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html   

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html   


###  20 News Groups Corpus, Sample dataset included in scikit-learn
A collection of almost 20,000 articles on 20 different topics or 'newsgroups'.   
Corpus: Text Collection

In [1]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

In [7]:
# first 3 classes (of 20)
twenty_train.target_names[0:3]
# python indexing excludes end index

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc']

In [17]:
# data and target
# has the input and desired output
# 11K of them are split for training pairs

print( len(twenty_train.data) )
print( len(twenty_train.target) )

11314
11314


In [30]:
i = 29
print(twenty_train.data[i])

From: jimf@centerline.com (Jim Frost)
Subject: Re: Is car saftey important?
Organization: CenterLine Software, Inc.
Lines: 14
NNTP-Posting-Host: 140.239.3.202

tcorkum@bnr.ca (Trevor Corkum) writes:
>Is it only me, or is
>safety not one of the most important factors when buying a car?

It depends on your priorities.  A lot of people put higher priorities
on gas mileage and cost than on safety, buying "unsafe" econoboxes
instead of Volvos.  I personally take a middle ground -- the only
thing I really look for is a three-point seatbelt and 5+mph bumpers.
I figure that 30mph collisions into brick walls aren't common enough
for me to spend that much extra money for protection, but there are
lots of low-speed collisions that do worry me.

jim frost
jimf@centerline.com



In [31]:
twenty_train.target[i]

7

In [32]:
twenty_train.target_names[  twenty_train.target[i]  ]

'rec.autos'

## scikit-learn Pipeline
- Scitkit-learn pipelines are a sequence of transforms followed by a final estimator.   
- Intermediate steps within the pipeline must be ‘transforms’ 
 * they must implement fit and transform methods 
 * The CountVectorizer and TfidfTransformer are transformers in this example   
- The estimator of a pipeline, the final step, only needs to implement the fit method   

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

pipe = Pipeline([('vect', CountVectorizer(stop_words='english')),
                  ('tfidf', TfidfTransformer()),
                  ('model', MultinomialNB()),])

mod = pipe.fit(twenty_train.data, twenty_train.target)

predicted = mod.predict(twenty_test.data)

print(classification_report(twenty_test.target,
                            predicted, 
                            target_names=twenty_test.target_names))

from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(twenty_test.target, predicted))

                          precision    recall  f1-score   support

             alt.atheism       0.80      0.69      0.74       319
           comp.graphics       0.78      0.72      0.75       389
 comp.os.ms-windows.misc       0.79      0.72      0.75       394
comp.sys.ibm.pc.hardware       0.68      0.81      0.74       392
   comp.sys.mac.hardware       0.86      0.81      0.84       385
          comp.windows.x       0.87      0.78      0.82       395
            misc.forsale       0.87      0.80      0.83       390
               rec.autos       0.88      0.91      0.90       396
         rec.motorcycles       0.93      0.96      0.95       398
      rec.sport.baseball       0.91      0.92      0.92       397
        rec.sport.hockey       0.88      0.98      0.93       399
               sci.crypt       0.75      0.96      0.84       396
         sci.electronics       0.84      0.65      0.74       393
                 sci.med       0.92      0.79      0.85       396
         

In [42]:
# So easy to add, remove, or modify steps and retest 
pipe = Pipeline([('vect', CountVectorizer(stop_words='english')),
                  #('tfidf', TfidfTransformer()),
                  ('model', MultinomialNB()),])

mod = pipe.fit(twenty_train.data, twenty_train.target)

predicted = mod.predict(twenty_test.data)

print(classification_report(twenty_test.target,
                            predicted, 
                            target_names=twenty_test.target_names))

from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(twenty_test.target, predicted))

                          precision    recall  f1-score   support

             alt.atheism       0.80      0.81      0.80       319
           comp.graphics       0.65      0.80      0.72       389
 comp.os.ms-windows.misc       0.80      0.04      0.08       394
comp.sys.ibm.pc.hardware       0.55      0.80      0.65       392
   comp.sys.mac.hardware       0.85      0.79      0.82       385
          comp.windows.x       0.69      0.84      0.76       395
            misc.forsale       0.89      0.74      0.81       390
               rec.autos       0.89      0.92      0.91       396
         rec.motorcycles       0.95      0.94      0.95       398
      rec.sport.baseball       0.95      0.92      0.93       397
        rec.sport.hockey       0.92      0.97      0.94       399
               sci.crypt       0.80      0.96      0.87       396
         sci.electronics       0.79      0.70      0.74       393
                 sci.med       0.88      0.87      0.87       396
         

### TRY: remove stop_words removal and see change in accuracy

## Experimenting, and Hyperparameter tuning using GridSearchCV and Pipeline
Similar to testing whether to remove stopwords or not, we want to run many experiments, with many combinations of parameters.  
GridSearchCV does this, for an estimator, or on a full pipeline. 
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html   
   
For  setting parameters of the various steps in the 
pipeline, you use <step name>+"__"+<parameter name>, for the list of possible values you want to test.

In [47]:
# Configure experiments
    
from sklearn.model_selection import GridSearchCV
parameters = {'vect__stop_words': ('english', None)}

# We can perform the grid search using all available CPU by setting n_jobs=-1:
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1)

In [49]:
# Run the experiments 
gs_model = grid_search.fit(twenty_train.data, twenty_train.target)

In [50]:
# Results of 
gs_model.cv_results_

{'mean_fit_time': array([4.31530085, 4.42989559]),
 'std_fit_time': array([0.1378511 , 0.07756394]),
 'mean_score_time': array([0.87700033, 0.66064115]),
 'std_score_time': array([0.0977623, 0.0908348]),
 'param_vect__stop_words': masked_array(data=['english', None],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'vect__stop_words': 'english'}, {'vect__stop_words': None}],
 'split0_test_score': array([0.86866461, 0.84045835]),
 'split1_test_score': array([0.86981465, 0.84112974]),
 'split2_test_score': array([0.85468198, 0.82773852]),
 'split3_test_score': array([0.8619469 , 0.83362832]),
 'split4_test_score': array([0.86784922, 0.83991131]),
 'mean_test_score': array([0.86459254, 0.83657416]),
 'std_test_score': array([0.00565275, 0.00517274]),
 'rank_test_score': array([1, 2], dtype=int32)}

In [ ]:
parameters = {'tfidf__use_idf': (True, False),
              'model__alpha': (1e1, 1e-3),
              'model__fit_prior': (True,False)}


grid_search = GridSearchCV(pipe, parameters, n_jobs=-1)